In [1]:
import os
import swiftclient.client
config = {'user':os.environ['OS_USERNAME'], 
          'key':os.environ['OS_PASSWORD'],
          'tenant_name':os.environ['OS_TENANT_NAME'],
          'authurl':os.environ['OS_AUTH_URL']}

conn = swiftclient.client.Connection(auth_version=2, **config)


In [2]:
config = {'username':os.environ['OS_USERNAME'], 
          'api_key':os.environ['OS_PASSWORD'],
          'project_id':os.environ['OS_TENANT_NAME'],
          'auth_url':os.environ['OS_AUTH_URL'],
           }
from novaclient.client import Client
nc = Client('2',**config)

In [3]:
# List containers
cont = 'tweets'
(response, bucket_list) = conn.get_account()
(response, bucket) = conn.get_container(cont)
filenames = [obj['name'] for obj in bucket]



/Library/Python/2.7/site-packages/keystoneclient/service_catalog.py:196: UserWarning: Providing attr without filter_value to get_urls() is deprecated as of the 1.7.0 release and may be removed in the 2.0.0 release. Either both should be provided or neither should be provided.
  'Providing attr without filter_value to get_urls() is '


In [ ]:
#searchwords = [('han',0),('hon',0),('den',0),('det',0),('denna',0),('denne',0),('hen',0)]
searchwords = ['han','hon','den','det','denna','denne','hen']

In [4]:
serverName = 'ChocolateFactory'
aKey = 'aKey'
keyPair = nc.keypairs.find(name = aKey)
flavor =  nc.flavors.find(name = 'm1.medium')
image = nc.images.find(name = 'Ubuntu Server 14.04 LTS (Trusty Tahr)')
print keyPair.name, flavor.name, image.name

aKey m1.medium Ubuntu Server 14.04 LTS (Trusty Tahr)


In [5]:
try:
    server = nc.servers.find(name = serverName)
except Exception as e:
    print e
    try:
        server = nc.servers.create(serverName,
                                   image,
                                   flavor,
                                   key_name = keyPair.name
                                  )
    except Exception as e:
        raise Exception("Failed to create server.\n{0}".format(e))
    pass
print server.name



ChocolateFactory


In [6]:
try:
    floating_ip = [ip for ip in nc.floating_ips.list() if ip.instance_id == server.id ].pop().ip
    print floating_ip
except:
    pass


130.238.29.90


In [ ]:
floating_ip = None
try:
    try:
        floating_ip = [ip for ip in nc.floating_ips.list() if ip.instance_id == server.id ].pop()
        print floating_ip.ip
    except Exception:
        pass
    if floating_ip is None :
        floating_ip = nc.floating_ips.create('ext-net')
    server.add_floating_ip(floating_ip.ip)
    print "ip={0}".format(floating_ip.ip)
except Exception as e:
    raise Exception("Failed to attach a floating IP to the controller.\n{0}".format(e))

In [ ]:
import paramiko
from paramiko import SSHClient as client 
ip = floating_ip.ip
try:
    ssh = client()
    ssh.set_missing_host_key_policy(
       paramiko.AutoAddPolicy())
    ssh.connect(ip,
            username = 'ubuntu',
            key_filename ='akey.pem')
except Exception as e:
    print e
commands = [
            'sudo apt-get update -y',
            'sudo apt-get install git -y',
            'sudo apt-get install python-pip -y',
            'sudo pip install Flask',
            'sudo apt-get install rabbitmq-server -y',
            'sudo pip install celery',
            'git clone https://github.com/MaxAndersson/pyTasks.git',
            'cd pyTasks',
            'celery worker -A tasks'
            'python app.py > /dev/null 2>&1 &'
           ]
for command in commands:
    try:
        print 'EXECUTEING: {}'.format(command)
        (stdin,stdout,stderr) = ssh.exec_command(command)
        print stdout.read()
        print stderr.read()
        
    except Exception as e:
        print e

In [ ]:
 a =  [(file['name'],file['bytes']) for file in bucket ]

for line in a:
    print line[0],'size:',line[1]/2**20, 'Mb'

    #conn.service.download(cont,['tweets_19.txt'])


        

In [ ]:
BROKER_URL = 'amqp://guest:guest@{}:5672//'.format(ip.ip)
print BROKER_URL

In [ ]:
import json
import numpy as np

searchwords = [['han',0],['hon',0],['den',0],['det',0],['denna',0],['denne',0],['hen',0]]


tweets = np.array([])

#rows = len(tweets)
#print rows
#for i in range(0,rows):
with open('tweets_19.txt') as lines:
    for line in lines:
        if line != '\n':
            tweets = np.append(tweets,json.loads(line))
    
for tweet in tweets:
    if tweet['text'][:2] == 'RT':
        print tweet['text']
        for word in searchwords:
            aCount = tweet['text'].count(word[0])
            if aCount != 0:
                word[1] = word[1] + aCount
                
                
print searchwords


In [ ]:
import urllib2
def downloadFile(name):
    url = "http://smog.uppmax.uu.se:8080/swift/v1/tweets/{}".format(name)

    file_name = url.split('/')[-1]
    u = urllib2.urlopen(url)
    f = open(file_name, 'wb')
    meta = u.info()
    file_size = int(meta.getheaders("Content-Length")[0])
    print "Downloading: %s Bytes: %s" % (file_name, file_size)

    file_size_dl = 0
    block_sz = 8192
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break

        file_size_dl += len(buffer)
        f.write(buffer)
        status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
        status = status + chr(8)*(len(status)+1)
        print status,

    f.close()
    return file_name

In [ ]:
import json
def countMentionInTweetFile(aFile,words):
    
    searchwords = [[word, 0] for word in words]
   
        #downloadFile(aFile),'r')
    with open(aFile,'r') as lines:
        for line in lines:
            if line != '/n':
                tweet = json.loads(line)
          
                if tweet['text'][:2] == 'RT':
                    #print tweet['text']
                    for word in searchwords:
                        word[1] = word[1] + tweet['text'].count(word[0])
        
    return searchwords

In [ ]:
import json
def countMentionInTweetFile(aFile,words):
    searchwords = [[word, 0] for word in words]

    with open(aFile,'r') as lines:
        for line in lines:
            if line != '\n':
                tweet = json.loads(line)
                if tweet['text'][:2] != 'RT':
                    #print tweet['text']
                    for word in searchwords:
                        word[1] = word[1] + tweet['text'].count(word[0])
    return json.dumps(dict(searchwords), indent = 4)

In [ ]:
searchwords = ['han','hon','den','det','denna','denne','hen']
print countMentionInTweetFile('tweets_19.txt',searchwords)


In [ ]:
# Terminate all your running instances
server = nc.servers.find(name = serverName)
server.delete()
print server